It is your job to predict the sales price for each house. 
For each Id in the test set, you must predict the value of the SalePrice variable. 

Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. 
https://en.wikipedia.org/wiki/Root-mean-square_deviation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn import metrics


import warnings
warnings.simplefilter("ignore")

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv') 
df_train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Baseline

In [77]:
# Попробуем 'тупую' модель только на выбранный скоррелированных данных

In [78]:
clf_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=1, random_state = rs)
clf_tree_reg = tree.DecisionTreeRegressor(max_depth=5, random_state = rs)
clf_Ridge = linear_model.Ridge(alpha=1, random_state = rs)
clf_Lasso= linear_model.Lasso(alpha=1, random_state = rs)
clf_linear = linear_model.LinearRegression()

# clf_log_reg = linear_model.LogisticRegression('l2')
list_clf = [clf_tree, clf_tree_reg, clf_Ridge, clf_Lasso, clf_linear]

In [79]:
def baseline_fit(clf, X=X_train, y=y_train, X_t=X_test, y_t=y_test):
    clf.fit(X, y)
    print(clf.score(X_t, y_t))
    print(metrics.mean_squared_error(y_t, clf.predict(X_t), squared=False))


In [80]:
for i in list_clf:
    print(i)
    baseline_fit(i)
    print(' ')

DecisionTreeClassifier(criterion='entropy', max_depth=1, random_state=5)
0.012448132780082987
57806.58332093901
 
DecisionTreeRegressor(max_depth=5, random_state=5)
0.7434174816500432
38017.28755187841
 
Ridge(alpha=1, random_state=5)
0.7890412819419222
34471.9883924821
 
Lasso(alpha=1, random_state=5)
0.7888781365707901
34485.315308974205
 
LinearRegression()
0.7888590662522396
34486.872776986296
 


In [26]:
df_train.GarageYrBlt = df_train.GarageYrBlt.fillna(0)

In [49]:
df_train.isna().sum().sum()

6884

In [30]:
type(int(df_train.GarageYrBlt[0]))

int

In [61]:
def fillna_df(x):
    list_col = x.columns
    for i in list_col:
        try:
            if type(int(x[i].mean())) == int:
                col_mean = x[i].mean()
                x[i] = x[i].fillna(col_mean)
                print(f'{i} - True')
        except:
            print(f'{i} - False')
            continue
    return x
        

In [62]:
new_df = fillna_df(df_train)

Id - True
MSSubClass - True
MSZoning - False
LotFrontage - True
LotArea - True
Street - False
Alley - False
LotShape - False
LandContour - False
Utilities - False
LotConfig - False
LandSlope - False
Neighborhood - False
Condition1 - False
Condition2 - False
BldgType - False
HouseStyle - False
OverallQual - True
OverallCond - True
YearBuilt - True
YearRemodAdd - True
RoofStyle - False
RoofMatl - False
Exterior1st - False
Exterior2nd - False
MasVnrType - False
MasVnrArea - True
ExterQual - False
ExterCond - False
Foundation - False
BsmtQual - False
BsmtCond - False
BsmtExposure - False
BsmtFinType1 - False
BsmtFinSF1 - True
BsmtFinType2 - False
BsmtFinSF2 - True
BsmtUnfSF - True
TotalBsmtSF - True
Heating - False
HeatingQC - False
CentralAir - False
Electrical - False
1stFlrSF - True
2ndFlrSF - True
LowQualFinSF - True
GrLivArea - True
BsmtFullBath - True
BsmtHalfBath - True
FullBath - True
HalfBath - True
BedroomAbvGr - True
KitchenAbvGr - True
KitchenQual - False
TotRmsAbvGrd - True
Fu

In [63]:
new_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [64]:
new_df.isna().sum().sum()

6617